In [1]:
!pip install ortools folium geopy pandas

In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('archive.zip')  # Make sure the file name matches exactly

# Print the first 5 rows to verify
print(df.head())

        Order_ID  Agent_Age  Agent_Rating  Store_Latitude  Store_Longitude  \
0  ialx566343618         37           4.9       22.745049        75.892471   
1  akqg208421122         34           4.5       12.913041        77.683237   
2  njpu434582536         23           4.4       12.914264        77.678400   
3  rjto796129700         38           4.7       11.003669        76.976494   
4  zguw716275638         32           4.6       12.972793        80.249982   

   Drop_Latitude  Drop_Longitude  Order_Date Order_Time Pickup_Time  \
0      22.765049       75.912471  2022-03-19   11:30:00    11:45:00   
1      13.043041       77.813237  2022-03-25   19:45:00    19:50:00   
2      12.924264       77.688400  2022-03-19   08:30:00    08:45:00   
3      11.053669       77.026494  2022-04-05   18:00:00    18:10:00   
4      13.012793       80.289982  2022-03-26   13:30:00    13:45:00   

      Weather  Traffic      Vehicle            Area  Delivery_Time  \
0       Sunny    High   motorcycle

In [5]:
# Cell A – Clean data + add depot (run this first)
import pandas as pd

# Your original file had Drop_Latitude / Drop_Longitude → rename them
df = df.rename(columns={'Drop_Latitude': 'lat', 'Drop_Longitude': 'lon'})

# Create a fake depot (average of first 20 drops – looks realistic)
depot_lat = df['lat'].head(20).mean()
depot_lon = df['lon'].head(20).mean()

# Insert depot as row 0
depot_row = pd.DataFrame({'Order_ID': ['DEPOT'], 'lat': [depot_lat], 'lon': [depot_lon]})
df = pd.concat([depot_row, df.head(100)], ignore_index=True)  # First 100 stops for speed

print(f"Ready: {len(df)-1} delivery stops + 1 depot")
print(df.head())

Ready: 100 delivery stops + 1 depot
        Order_ID        lat        lon  Agent_Age  Agent_Rating  \
0          DEPOT  18.471778  77.851781        NaN           NaN   
1  ialx566343618  22.765049  75.912471       37.0           4.9   
2  akqg208421122  13.043041  77.813237       34.0           4.5   
3  njpu434582536  12.924264  77.688400       23.0           4.4   
4  rjto796129700  11.053669  77.026494       38.0           4.7   

   Store_Latitude  Store_Longitude  Order_Date Order_Time Pickup_Time  \
0             NaN              NaN         NaN        NaN         NaN   
1       22.745049        75.892471  2022-03-19   11:30:00    11:45:00   
2       12.913041        77.683237  2022-03-25   19:45:00    19:50:00   
3       12.914264        77.678400  2022-03-19   08:30:00    08:45:00   
4       11.003669        76.976494  2022-04-05   18:00:00    18:10:00   

      Weather  Traffic      Vehicle            Area  Delivery_Time  \
0         NaN      NaN          NaN             NaN 

In [7]:
# Cell C – Baseline distance
from geopy.distance import geodesic

total = 0
for i in range(len(df)-1):
    a = (df.iloc[i]['lat'], df.iloc[i]['lon'])
    b = (df.iloc[i+1]['lat'], df.iloc[i+1]['lon'])
    total += geodesic(a, b).miles

print(f"Baseline unoptimized distance: {total:.1f} miles")

Baseline unoptimized distance: 128960.9 miles


In [9]:
# Cell: Clean bad coordinates (removes anything outside India)
print("Rows before cleaning:", len(df))

# India is roughly latitude 8–37, longitude 68–89
df_clean = df[
    (df['lat'].between(8, 37)) &
    (df['lon'].between(68, 89))
].copy()

print("Rows after cleaning bad coordinates:", len(df_clean))
print("Removed", len(df) - len(df_clean), "ocean points")

# Use the cleaned version from now on
df = df_clean.reset_index(drop=True)
df.head()

Rows before cleaning: 101
Rows after cleaning bad coordinates: 91
Removed 10 ocean points


/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")
/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Order_ID,lat,lon,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,DEPOT,18.471778,77.851781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ialx566343618,22.765049,75.912471,37.0,4.9,22.745049,75.892471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120.0,Clothing
2,akqg208421122,13.043041,77.813237,34.0,4.5,12.913041,77.683237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165.0,Electronics
3,njpu434582536,12.924264,77.688400,23.0,4.4,12.914264,77.678400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130.0,Sports
4,rjto796129700,11.053669,77.026494,38.0,4.7,11.003669,76.976494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105.0,Cosmetics


In [12]:
import folium

# Clean India-only points (just in case)
df = df[(df['lat'].between(6, 38)) & (df['lon'].between(68, 89))].reset_index(drop=True)

# Depot = average of all good points
depot_lat = df['lat'].mean()
depot_lon = df['lon'].mean()

# THIS LINE FORCES ENGLISH + beautiful Google-style map
m = folium.Map(location=[depot_lat, depot_lon], zoom_start=6, tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='Google')

# Depot marker
folium.Marker([depot_lat, depot_lon], popup="Amazon Depot",
              icon=folium.Icon(color="blue", icon="warehouse", prefix='fa')).add_to(m)

# Customer stops
for i in range(len(df)):
    folium.CircleMarker([df.iloc[i]['lat'], df.iloc[i]['lon']], radius=5, color="red", fill=True,
                        popup=f"Stop {i}").add_to(m)

# Gray spaghetti
for i in range(len(df)-1):
    folium.PolyLine([[df.iloc[i]['lat'], df.iloc[i]['lon']],
                     [df.iloc[i+1]['lat'], df.iloc[i+1]['lon']]],
                    color="gray", weight=3, opacity=0.8).add_to(m)



In [17]:
# FINAL FINAL — Beautiful multi-van rainbow map (copy & run once)
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from geopy.distance import geodesic
import folium
import numpy as np

# Distance matrix
print("Building distance matrix...")
n = len(df)
dist_matrix = np.zeros((n, n), dtype=int)
for i in range(n):
    for j in range(i+1, n):
        d = int(geodesic((df.iloc[i]['lat'], df.iloc[i]['lon']),
                         (df.iloc[j]['lat'], df.iloc[j]['lon'])).meters)
        dist_matrix[i][j] = dist_matrix[j][i] = d

# Solver
manager = pywrapcp.RoutingIndexManager(n, 15, 0)
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node   = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# ADD THIS BLOCK — forces 8–12 vans with max 15 stops each
routing.AddConstantDimension(1, 15, True, "Capacity")
demand_callback = routing.RegisterUnaryTransitCallback(lambda x: 1)
routing.AddDimensionWithVehicleCapacity(demand_callback, 0, [15]*15, True, "Capacity")

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

print("Optimizing with 8–12 vans...")
solution = routing.SolveWithParameters(search_parameters)

# Extract & print results
total_meters = solution.ObjectiveValue() if solution else 0
total_miles = total_meters / 1609.34
savings = (1 - total_miles / total) * 100

print(f"\nOPTIMIZED DISTANCE : {total_miles:,.1f} miles")
print(f"BASELINE           : {total:,.1f} miles")
print(f"SAVINGS            : {savings:.1f}%")
print(f"NUMBER OF VANS USED: {sum(1 for v in range(15) if not routing.IsVehicleUsed(solution, v) == 0)}")

# Draw rainbow routes
m_after = folium.Map(location=[depot_lat, depot_lon], zoom_start=6,
                     tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='Google')
folium.Marker([depot_lat, depot_lon], popup="Amazon Depot", icon=folium.Icon(color="blue", icon="warehouse", prefix='fa')).add_to(m_after)

colors = ['green','purple','orange','darkred','lightblue','pink','beige','darkgreen','cadetblue','darkpurple','lightred','gray','black','lightgray','red']

for vehicle_id in range(15):
    index = routing.Start(vehicle_id)
    if routing.IsEnd(solution.Value(routing.NextVar(index))):
        continue
    route_points = []
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route_points.append([df.iloc[node]['lat'], df.iloc[node]['lon']])
        index = solution.Value(routing.NextVar(index))
    color = colors[vehicle_id % len(colors)]
    folium.PolyLine(route_points, color=color, weight=6, opacity=0.9).add_to(m_after)
    for lat, lon in route_points:
        folium.CircleMarker([lat, lon], radius=4, color=color, fill=True).add_to(m_after)

m_after

Building distance matrix...
Optimizing with 8–12 vans...

OPTIMIZED DISTANCE : 8,694.1 miles
BASELINE           : 128,960.9 miles
SAVINGS            : 93.3%
NUMBER OF VANS USED: 7


In [18]:
print(f"FINAL RESULTS")
print(f"Before optimization : {total:,.1f} miles")
print(f"After optimization  : {total_miles:,.1f} miles")
print(f"Miles saved         : {(total - total_miles):,.1f} miles")
print(f"Savings             : {((total - total_miles)/total*100):.1f}%")
print(f"Vans used           : {sum(1 for v in range(15) if routing.IsVehicleUsed(solution, v))}")

FINAL RESULTS
Before optimization : 128,960.9 miles
After optimization  : 8,694.1 miles
Miles saved         : 120,266.8 miles
Savings             : 93.3%
Vans used           : 7
